In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from keras.utils import np_utils

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import keras.optimizers


# path to save model
save_model_path ='\\Model\\'

# path to images
# name the subfolders like your classes
path_train_img = '\\trainingdata\\'
path_validation_img = '\\validationdata\\'

# set number of epochs and batch size 
epochs = 100
batch_size = 30

# define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(240,320,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))


# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# initiate my image data generators
train_img = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_img = ImageDataGenerator(rescale=1./255)

# prepare the data using the generators
train_generator = train_img.flow_from_directory(
    path_train_img,
    target_size=(240, 320),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_img.flow_from_directory(
    path_validation_img,
    target_size=(240, 320),
    batch_size=batch_size,
    class_mode='categorical')


# fit the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
    
# save model    
model.save(save_model_path+'MyModel')
model.save_weights('mymodel_weights.h5')
model.save('mymodel_keras.h5')

In [ ]:
# Accuracy
scores = model.evaluate(validation_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# plot accuracy curve and loss curve

import seaborn as sns
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(2,1,figsize=[8,8], sharex=True)

ax1.plot(history.history['accuracy'], color='green')
ax1.plot(history.history['val_accuracy'], color='cornflowerblue')
ax1.set_ylabel('Accuracy', fontsize=17)
#ax1.set_xlabel('Epoch', fontsize=17)
ax2.plot(history.history['loss'], color='green')
ax2.plot(history.history['val_loss'], color='cornflowerblue')
ax2.set_ylabel('Loss', fontsize=17)
ax2.set_xlabel('Epoch', fontsize=17)
#ax1.legend(['train', 'test'], loc='upper left')
#ax2.legend(['train', 'test'], loc='upper left')
sns.despine(top=True,bottom=True,right=True,left=True)
sns.set(style='white',rc={"lines.linewidth": 3})
ax1.tick_params(axis='y', labelsize=15, color='dimgrey')
ax1.tick_params(axis='x', labelsize=15, color='lightgrey')
ax2.tick_params(axis='x', labelsize=15, color='dimgrey')
ax2.tick_params(axis='y', labelsize=15, color='lightgrey')


fig.savefig('Model_Metrics.jpeg', bbox_inches='tight')

plt.show()

In [ ]:
# optional steps to add above compile if testing with callbacks

# This checkpoint object will store the model parameters # in the file "CozmoWeights.hdf5"
checkpoint = ModelCheckpoint('CozmoWeights.hdf5', monitor='val_accuracy', save_best_only=True)

# Store the checkpoint during training
cozmo_callbacks = [checkpoint]